## 実行方法

サイドメニューのFile BrowserからTwitterのアーカイブ（ZIPファイル）をtmpディレクトリに「アップロード」（実際にはローカルコピーであり、外部ネットワークへの送信ではありません）した後、最初のセルから順に実行してください。再生ボタン（「Run the selected cells and advance (Shift+Enter)」）を押下することで選択したセルを実行できます。処理対象（アーカイブ、ディレクトリ）を尋ねられるので都度選択してボタンを押下してください。

In [ ]:
# アーカイブを解凍する
import pathlib
import ipywidgets as widgets
from IPython.display import display
import shutil

tmp = pathlib.Path('tmp')
archive_files_before_sorting = [file.name for file in tmp.glob('*.zip')]
archive_files = sorted(archive_files_before_sorting, reverse=True)

if not archive_files:
    print("分析対象となるファイルが存在しませんでした。")
    raise KeyboardInterrupt
else:
    radioButtons_for_selecting_archive = widgets.RadioButtons(
        options=archive_files,
        layout={'width': 'max-content'},
        description="アーカイブファイル名選択",
        disabled=False
    )
    button = widgets.Button(description="解凍")
    output = widgets.Output()

    @output.capture(clear_output=True)
    def unpack_archive(clicked_button: widgets.Button) -> None:
        radioButtons_for_selecting_archive.disabled = True
        button.disabled = True

        archive_file = tmp / radioButtons_for_selecting_archive.value
        dst = tmp / archive_file.stem
        shutil.unpack_archive(archive_file, dst)

        radioButtons_for_selecting_archive.disabled = False
        button.disabled = False
        print("解凍しました。")

    button.on_click(unpack_archive)
    display(radioButtons_for_selecting_archive, button, output)

    if len(archive_files) == 1:
        unpack_archive(button)

In [ ]:
# 解凍されたアーカイブを分析する
import pathlib
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import json
import pandas as pd
import matplotlib.pyplot as plt
import warnings

tmp = pathlib.Path('tmp')
analysis_target_directories_before_sorting = [target_dir.name for target_dir in tmp.iterdir() if target_dir.is_dir()]
analysis_target_directories = sorted(analysis_target_directories_before_sorting, reverse=True)

if not analysis_target_directories:
    print("分析対象となるファイルが存在しませんでした。")
    raise KeyboardInterrupt
else:
    radioButtons_for_selecting_analysis_target = widgets.RadioButtons(
        options=analysis_target_directories,
        layout={'width': 'max-content'},
        description="分析対象ディレクトリ名選択",
        disabled=False
    )
    button = widgets.Button(description="分析")
    output = widgets.Output()

    @output.capture(clear_output=True)
    def analyse(clicked_button: widgets.Button) -> None:
        radioButtons_for_selecting_analysis_target.disabled = True
        button.disabled = True

        analysis_target_directory = radioButtons_for_selecting_analysis_target.value
        read_path = tmp / analysis_target_directory / 'data' / 'tweets.js'

        raw_file_content = read_path.read_text(encoding='utf-8')
        json_formatted_file_content = raw_file_content.replace('window.YTD.tweets.part0 = ', '', 1)
        df = pd.json_normalize(json.loads(json_formatted_file_content))

        df['tweet.created_at'] = pd.to_datetime(df['tweet.created_at'], format='%a %b %d %H:%M:%S %z %Y').dt.tz_convert('Asia/Tokyo')

        df['year'] = df['tweet.created_at'].dt.year
        df['month'] = df['tweet.created_at'].dt.month
        df['day'] = df['tweet.created_at'].dt.day
        df['weekday'] = df['tweet.created_at'].dt.weekday
        df['hour'] = df['tweet.created_at'].dt.hour

        def aggregate_tweets_by(period):
            size_by_period = df.groupby(period).size()
            fig, ax = plt.subplots()
            size_by_period.plot(ax=ax)
            plt.setp(ax.get_xticklabels(), rotation=60, ha="right")
            plt.show()
            print(size_by_period)


        interact(aggregate_tweets_by, period=[("年別", 'year'),("月別", ['year', 'month']),("日別", ['year', 'month', 'day']),("曜日別", 'weekday'),("時間帯別", 'hour'),("曜日時間帯別", ['weekday', 'hour'])])

        radioButtons_for_selecting_analysis_target.disabled = False
        button.disabled = False

    button.on_click(analyse)
    display(radioButtons_for_selecting_analysis_target, button, output)

    if len(analysis_target_directories) == 1:
        analyse(button)